In [ ]:
import mlflow
from mlflow.tracking import MlflowClient
from model import APP_NAME, ROUTE_PREFIX, MODEL_NAME, MODEL_VERSION, MLFLOW_URI

FILE_NAME = "model.py"

# 1. Conectar a MLflow y obtener el Run ID del modelo
mlflow.set_tracking_uri(MLFLOW_URI)
client = MlflowClient()

# Obtenemos la info de la versión registrada para saber su Run ID origen
mv = client.get_model_version(MODEL_NAME, MODEL_VERSION)
run_id = mv.run_id
print(f"🔹 Subiendo artefactos al Run ID: {run_id}")

# 2. Subir model.py al Run
client.log_artifact(run_id, FILE_NAME, artifact_path="deployment")

# 3. Crear y subir metadata del despliegue (nombre y ruta)
deploy_meta = {
    "app_name": APP_NAME,
    "route_prefix": ROUTE_PREFIX,
    "file_name": FILE_NAME,
    "run_id": run_id
}
client.log_dict(run_id, deploy_meta, "deployment/deploy_config.json")
print("✅ model.py y configuración guardados en MLflow.")


# --- Gestión de Tags (Exclusividad) ---
TAG_KEY = "backup"
TAG_VALUE = "active"

# 1. Remover el tag de todas las otras versiones
all_versions = client.search_model_versions(f"name='{MODEL_NAME}'")
for version in all_versions:
    if version.version != MODEL_VERSION:
        # Intentamos borrar el tag, si no existe no pasa nada crítico
        try:
            client.delete_model_version_tag(MODEL_NAME, version.version, TAG_KEY)
        except Exception:
            pass

# 2. Asignar el tag a la versión actual
client.set_model_version_tag(MODEL_NAME, MODEL_VERSION, TAG_KEY, TAG_VALUE)

print(f"✅ Tag '{TAG_KEY}={TAG_VALUE}' establecido en v{MODEL_VERSION} y removido de versiones anteriores.")
